In [1]:
import numpy as np
from random import random


## Κλάση πολυεπίπεδου αισθητήρα

In [2]:
# Ορισμός Κλάσης
class MLP(object):

    # Αρχικοποίηση κλάσης - Με ορίσματα τον αριθμό των εισόδων, κρυφών επιπέδον και εξόδων.
    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # Δημιουργία γενικής αναπαράστασης επιπέδων
        layers = [num_inputs] + hidden_layers + [num_outputs]
      
        # Δημιουργία τυχαίων βαρών για όλα τα επίπεδα
        # Κενή λίστα βαρών
        weights = []
        # Βρόχος επανάληψης για όλα τα επίπεδα
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights
        print(weights)

        # Παράγωγοι για όλα τα επίπεδα
        # Κενή λίστα παραγώγων
        derivatives = []
        # Βρόχος επανάληψης για όλα τα επίπεδα
        for i in range(len(layers) - 1):
          # Αρχικά όλες οι τιμές είναι 0
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives

        # Κατώφλι ενεργοποίησης για όλα τα επίπεδα
        # Κενή λίστα κατωφλιών
        activations = []
        # Βρόχος επανάληψης για όλα τα επίπεδα
        for i in range(len(layers)):
          # Αρχικά όλες οι τιμές είναι 0
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations


    def forward_propagate(self, inputs):

        # Για το επίπεδο των εισόδων οι ενεργοποίησεις ισούνται με τις εισόδους 
        activations = inputs

        # αποθήκευσης τιμών ενεργοποίησης
        self.activations[0] = activations

        # Επαναληπτικός βρόχος επεξεργασίας όλων των επιπέδων 
        for i, w in enumerate(self.weights):
          
            # Οι είσοδοι του επιπέδου προκύπτουν από τον πολλαπλασιασμό των τιμών ενεργοποίσης
            # του προηγούμενου επιπέδου με τον πίνακα βαρών
            net_inputs = np.dot(activations, w)

            # Εφαρμογή σιγμοειδούς συνάρτησης ενεργοποίησης
            activations = self._sigmoid(net_inputs)

            # Αποθήκευση ( θα χρησιμοποιηθεί στην οπίσθια τροφοδότηση)
            self.activations[i + 1] = activations

        # Επιστροφή της τιμής της εξόδου
        return activations


    def back_propagate(self, error):

        # Επαναληπτικός βρόχος επεξεργασίας όλων των επιπέδων (από δεξιά προς τα αριστερά)
        for i in reversed(range(len(self.derivatives))):

            # Τιμές ενεργοποίησης προηγούμενου επιπέδου
            activations = self.activations[i+1]

            # Παραγώγηση των τιμών ενεργοποίησης
            delta = error * self._sigmoid_derivative(activations)

            # Υπολογισμός ανάστροφου πίνακα
            delta_re = delta.reshape(delta.shape[0], -1).T

            # Τιμές ενεργοποίησης τρέχοντος επιπέδου
            current_activations = self.activations[i]

            # Υπολογισμός ανάστροφου πίνακα
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # Υπολογισμός κλίσης
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # Οπισθοδιάδωση σφάλματος
            error = np.dot(delta, self.weights[i].T)


    def train(self, inputs, targets, epochs, learning_rate):

        # Επαναληπτικός βρόχος εκπαίδευσης ( πλήθος επαναλήψεων = εποχές )
        for i in range(epochs):
            sum_errors = 0

            # Επαναληπτικός βρόχος δεδομένων
            for j, input in enumerate(inputs):
                target = targets[j]

                # Εμπρόσθια τροφοδότηση
                output = self.forward_propagate(input)

                # Υπολογισμός σφάλματος
                error = target - output

                # Οπίσθοδιάδωση σφάλματος
                self.back_propagate(error)

                # Ενημέρωση βαρών με χρήση αλγορίθμου μετατροπής σύμφωνα με την κλίση
                self.gradient_descent(learning_rate)

                # Υπολογισμός μέσου τετραγωνικού σφάλματος
                sum_errors += self._mse(target, output)

            # Σφάλμα ανά εποχή
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")


    def gradient_descent(self, learningRate=1):

        # Ενημέρωση βαρών 
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate


    def _sigmoid(self, x):

        y = 1.0 / (1 + np.exp(-x))
        return y


    def _sigmoid_derivative(self, x):
        return x * (1.0 - x)


    def _mse(self, target, output):

        return np.average((target - output) ** 2)


## Προετοιμασία δεδομένων XOR


In [3]:
# Δείκτης ποσότητας δεδομένων - Θα καθορίσει το σύνολο των γραμμών του πίνακα
data_train_index = 4000

# Κενή λίστα, θα περιέχει τα δεδομένα εισόδου
items = []

# Βρόχος επανάληψης δημιοργίας τυχαίαων ζευγαριών με τιμές 0 ή 1
for i in range(data_train_index):
    item =np.random.choice([0, 1], size=2)
    items.append(item)

# Μετατροπή της λίστα σε πίνακα
items=np.array(items)

In [4]:
# Επαλήθευση πλήθους γραμμών πίνακας
len(items)

4000

In [5]:
# Προεπισκόπηση των πρώτων και τελευταίων πεδίων του πίνακα
print(items)

[[0 1]
 [0 0]
 [1 1]
 ...
 [1 0]
 [1 0]
 [0 0]]


In [6]:
# Κενή λίστα, θα περιέχει τα επιθυμητά δεδομένα εξόδου
targets = []

# Βρόχος επανάληψης - Ανάγνωση όλων των ζευγραριών από πίνακα items
for i in (items):
    for j in range(1):
      # Η επιθύμητη έξοδος είναι 1
        target=1
        # Αλλά αν οι είσοδοι είναι ίσες (0,0) ή (1,1)
        if i[j] == i[j+1]:
          # Η επιθυμητή έξοδος είναι 0
            target = 0
        targets.append(target)

In [7]:
# Επαλήθευση πλήθους εξόδων - Πρέπει να είναι όσα και τα ζεύγη
len(targets)

4000

In [32]:
# Προβολή 5 πρώτων και 5 τελευταίων πεδίων πίνακα επιθυμητών αποτελεσμάτων
print(targets[:5])
print(targets[-5:])

[1, 0, 0, 0, 1]
[1, 1, 1, 1, 0]


# Κύρια συνάρτηση (main)

In [9]:
# Δημιουργία πολυεπίπεδου αισθητήρα με 1 επίπεδο εισόδου, 1 κρυφό επίπεδο, 1 επίπεδο εξόδου 
mlp = MLP(2, [3], 1)

# Εκπαίδευση μοτνέλου
mlp.train(items, targets, 50, 0.1)

# Δεδομένα ελέγχου - Testing
input1 = np.array([1,1])
input2 = np.array([1,0])
input3 = np.array([0,1])
input4 = np.array([0,0])

# Πρόβλεψη για κάθε ζεύγος ελέγχου
output1 = mlp.forward_propagate(input1)
output2 = mlp.forward_propagate(input2)
output3 = mlp.forward_propagate(input3)
output4 = mlp.forward_propagate(input4)

# Εκτύπωση αποτελεσμάτων
print("*******************************************************")
print("Τhe MLP network believes that {} XOR {} is equal to {}".format(input1[0], input1[1], output1[0]))
print("Τhe MLP network believes that {} XOR {} is equal to {}".format(input2[0], input2[1], output2[0]))
print("Τhe MLP network believes that {} XOR {} is equal to {}".format(input3[0], input3[1], output3[0]))
print("Τhe MLP network believes that {} XOR {} is equal to {}".format(input4[0], input4[1], output4[0]))


[array([[0.07399649, 0.40639798, 0.14924643],
       [0.74981634, 0.15549161, 0.28068258]]), array([[0.68807024],
       [0.99568045],
       [0.44124621]])]
Error: 0.25312678406621836 at epoch 1
Error: 0.2509028533374089 at epoch 2
Error: 0.25087090405149093 at epoch 3
Error: 0.25084642458587353 at epoch 4
Error: 0.2508258112300136 at epoch 5
Error: 0.2508074446366243 at epoch 6
Error: 0.2507906100340191 at epoch 7
Error: 0.2507750249058143 at epoch 8
Error: 0.25076060823247687 at epoch 9
Error: 0.25074736234913936 at epoch 10
Error: 0.25073531275187805 at epoch 11
Error: 0.2507244765562421 at epoch 12
Error: 0.2507148417960369 at epoch 13
Error: 0.25070634798429636 at epoch 14
Error: 0.25069886452399387 at epoch 15
Error: 0.2506921650528926 at epoch 16
Error: 0.2506858893273449 at epoch 17
Error: 0.2506794656081704 at epoch 18
Error: 0.2506719220145897 at epoch 19
Error: 0.250661388948048 at epoch 20
Error: 0.2506436524936183 at epoch 21
Error: 0.250607219115712 at epoch 22
Error: 0.